In [7]:
import os
import numpy as np
import pickle
import torchaudio

In [24]:
sr = 16000
spec_window = 0.025
spec_shift = 0.010
chunk_size = int(10.26 * sr)

In [25]:
found_files = set([file.split(".")[0] for file in os.listdir("data_ogg/")])

In [26]:
path = "salami-data-public/annotations/"
annots = {}
for file in os.listdir(path):
    if file in found_files:
        try:
            with open(f"{path}{file}/parsed/textfile1_functions.txt", "r") as f:
                annots[file] = np.array([float(x.split("\t")[0]) for x in f.readlines()])
        except:
            with open(f"{path}{file}/parsed/textfile2_functions.txt", "r") as f:
                annots[file] = np.array([float(x.split("\t")[0]) for x in f.readlines()])

In [27]:
results = {}
for file in annots:
    print(file)
    end = int(np.ceil(sr * annots[file][-1])) + 1
    music, _ = torchaudio.load(f"data_ogg/{file}.ogg")[:end]
    labels = np.zeros(end).astype(int)
    labels[(annots[file] * sr).astype(int)[1: -1]] = 1
    shift = int(spec_shift * sr)
    w = int(spec_window * sr)
    labels2 = np.zeros(int(np.ceil((end - w) / shift))).astype(int)
    i = 0
    while i < len(labels2) :
        if any(labels[i * shift: i * shift + w]):
            labels2[i] = 1
        i += 1
    w = 1024
    shift = 10
    w_ = 16
    for i in range(0, len(labels2), w):
        labels3 = np.zeros(int(np.ceil((w - w_) / shift))).astype(int)
        j = 0
        while j < len(labels3):
            if any(labels2[i + j * shift: i + j * shift + w_]):
                labels3[j] = 1
            j += 1
        spectrogram = torchaudio.compliance.kaldi.fbank(music[..., int(i / w): int(i / w) + chunk_size], htk_compat=True, sample_frequency=16000, use_energy=False,
                                                  window_type='hanning', num_mel_bins=128, dither=0.0, frame_shift=10)
        results[f"{file}_{int(i / w)}"] = (spectrogram, labels3)

with open("train_data.pkl", "wb") as f:
    pickle.dump(results, f)

1069
1432
1264
1056
1290
1051
1263
1435
1058
1093
1403
1067
1299
1060
1404
1094
1034
1206
1450
1239
998
1468
1495
965
1237
1005
1461
991
1208
996
1230
962
1492
1459
1298
1095
1061
1253
1405
1059
1402
1254
1066
1092
1262
1050
1434
1296
1068
1291
963
1467
997
1231
1003
1458
1004
1236
990
1460
964
1494
1456
1032
1200
1469
999
955
1207
1035
1451
1238
1375
1147
1381
1386
1372
1149
1182
1176
1388
1343
1125
1317
1328
1310
1122
1326
1114
1319
1389
1184
1170
1342
1148
1183
1373
1141
1387
1179
1380
1146
1374
1112
1318
1123
1311
1316
1124
1139
1106
1334
1333
1101
1108
1130
1302
1190
1356
1164
1163
1351
1197
1155
1367
1358
1394
1360
1152
1199
1303
1131
1109
1136
1304
1100
1332
1138
1335
1107
1395
1198
1392
1366
1154
1359
1196
1350
1162
1165
1191
1368
1028
1487
983
1222
1474
984
970
1019
979
1442
1214
1026
1021
1213
1445
1247
1075
1278
1416
1072
1240
1086
1429
1044
1276
1420
1282
1285
1427
1271
1043
1088
1418
1212
1020
1444
1488
978
1018
1443
1027
1215
971
1011
1223
1475
1472
982
1224
1016
976
1042